# Main Original Code:

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rd

cols = ['Underlying']
strikeRange = [455, 456, 457, 458, 459, 460, 461]

for i in range(len(strikeRange)):
    cols += [f'Strike: ${strikeRange[i]}', f'call_openint {strikeRange[i]}', f'call_volume {strikeRange[i]}', f'call_bid {strikeRange[i]}',f'call_ask {strikeRange[i]}',f'put_bid {strikeRange[i]}',f'put_ask {strikeRange[i]}',f'put_volume {strikeRange[i]}',f'put_openint {strikeRange[i]}']

df = pd.DataFrame(columns=cols)

num_trades = 15
for t in range(num_trades):
    if t > 0:
        df.loc[t,'Underlying'] = df.loc[t-1,'Underlying'] + np.random.normal(0,5)
    else:
        df.loc[t,'Underlying'] = 458
        for strike in range(len(strikeRange)):
            df.loc[t,f'call_openint {strikeRange[strike]}'] = round(np.random.normal(100,30))
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(400 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            
        
    for strike in range(len(strikeRange)):
        df.loc[t,f'Strike: ${strikeRange[strike]}'] = f'Strike: ${strikeRange[strike]}'

        #set volume and open interest
        if t>0:
            df.loc[t,f'call_openint {strikeRange[strike]}'] = max(df.loc[t-1,f'call_openint {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            # if OI jumps and vol doesn't, market responds heavily
            # if OI jumps and vol jumps, market responds medium
            # if OI doesn't jump and vol does, market responds medium
            # if neither jump, status quo
            old_cOI = df.loc[t-1,f'call_openint {strikeRange[strike]}']
            old_pOI = df.loc[t-1,f'put_openint {strikeRange[strike]}']
            old_cVol = df.loc[t-1,f'call_volume {strikeRange[strike]}']
            old_pVol = df.loc[t-1,f'put_volume {strikeRange[strike]}']

            cOI = df.loc[t,f'call_openint {strikeRange[strike]}']
            pOI = df.loc[t,f'put_openint {strikeRange[strike]}']
            cVol = df.loc[t,f'call_volume {strikeRange[strike]}']
            pVol = df.loc[t,f'put_volume {strikeRange[strike]}']

            # if cOI > old_cOI + 5 and cVol < old_cVol + 5:
            #     # call jumps, put drops
            # elif pOI > old_pOI + 5 and pVol < old_pVol + 5:
            #     # call drops, put jumps
            # elif c

            
            #get call data
            call_bid = np.random.normal(2,1)
            call_ask = call_bid-np.random.normal(0.2,0.1) #
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            
            #calculate put call parity to back out put bid/ask
            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike]

            #get put data
            put_bid = mean_put+np.random.normal(0.2,0.1)
            put_ask = mean_put-np.random.normal(0.2,0.1)
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask
        
        else: #initial conditions
            #get call data
            call_bid = np.random.normal(2,1)
            call_ask = call_bid-np.random.normal(0.2,0.1)
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            
            #calculate put call parity to back out put bid/ask
            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike]

            #get put data
            put_bid = mean_put+np.random.normal(0.2,0.1)
            put_ask = mean_put-np.random.normal(0.2,0.1)
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask

#display(df)

# ADDED for debugging
#------------------

df = df.sort_values(by="Underlying").reset_index(drop=True)

cycle = 9
nCycles = len(strikeRange)
underlyingCol = df['Underlying']

df = df.iloc[:, 1:]

for i in range(0, nCycles):
    start = i*cycle
    end = start+cycle
    dfNew = df.iloc[:, start:end]
    dfNew.insert(loc=0, column = "Underlying", value=underlyingCol)
    display(dfNew)

#------------------

#so there are negative prices present and bid > ask

,Underlying,Strike: $455,call_openint 455,call_volume 455,call_bid 455,call_ask 455,put_bid 455,put_ask 455,put_volume 455,put_openint 455
0,452.921909,Strike: $455,112,38,1.750645,1.584173,4.034248,3.583433,62,88
1,453.227405,Strike: $455,102,48,1.880801,1.456283,3.587266,3.273081,52,98
2,456.021882,Strike: $455,103,47,1.893746,1.722777,0.865342,0.631214,53,97
3,456.255263,Strike: $455,97,53,0.708883,0.296635,-0.535657,-1.134464,47,103
4,457.185308,Strike: $455,111,39,0.5939,0.300463,-1.62584,-1.910438,61,89
5,458,Strike: $455,123,77,3.240116,2.932898,0.305306,-0.098823,0,277
6,458.193019,Strike: $455,115,35,3.174906,3.014134,-0.011963,-0.20283,65,85
7,458.906949,Strike: $455,114,36,0.39649,0.289262,-3.322802,-3.757979,64,86
8,460.207772,Strike: $455,109,41,2.180657,1.894439,-2.869007,-3.367868,59,91
9,460.499616,Strike: $455,126,24,0.997848,0.796177,-4.406368,-4.732153,76,74


,Underlying,Strike: $456,call_openint 456,call_volume 456,call_bid 456,call_ask 456,put_bid 456,put_ask 456,put_volume 456,put_openint 456
0,452.921909,Strike: $456,77,73,1.436206,1.155605,4.73271,4.215102,27,123
1,453.227405,Strike: $456,64,86,3.181841,2.758892,5.94904,5.551658,14,136
2,456.021882,Strike: $456,67,83,1.290967,1.247192,1.660886,0.975046,17,133
3,456.255263,Strike: $456,75,75,1.863405,1.639295,1.69293,1.135159,25,125
4,457.185308,Strike: $456,72,78,1.338716,1.211529,0.331714,-0.063939,22,128
5,458,Strike: $456,85,115,1.959695,1.927581,0.313645,-0.173416,0,315
6,458.193019,Strike: $456,63,87,3.155742,3.023559,1.057808,0.790099,13,137
7,458.906949,Strike: $456,69,81,0.752173,0.372221,-2.145051,-2.478704,19,131
8,460.207772,Strike: $456,73,77,2.001368,1.810858,-2.147934,-2.338952,23,127
9,460.499616,Strike: $456,88,62,1.193022,0.97583,-3.14843,-3.66288,38,112


,Underlying,Strike: $457,call_openint 457,call_volume 457,call_bid 457,call_ask 457,put_bid 457,put_ask 457,put_volume 457,put_openint 457
0,452.921909,Strike: $457,104,46,2.197266,1.975329,6.517362,5.886373,54,96
1,453.227405,Strike: $457,107,43,3.025407,2.804408,6.790318,6.6275,57,93
2,456.021882,Strike: $457,115,35,1.428618,1.478523,2.633068,2.166221,65,85
3,456.255263,Strike: $457,116,34,2.465095,2.221095,3.465471,2.819647,66,84
4,457.185308,Strike: $457,111,39,2.370152,2.123269,2.317403,1.8274,61,89
5,458,Strike: $457,106,94,3.194515,2.949303,2.192924,1.640614,0,294
6,458.193019,Strike: $457,102,48,1.400021,1.176638,0.478535,-0.051801,52,98
7,458.906949,Strike: $457,105,45,0.640267,0.248689,-1.226743,-1.616664,55,95
8,460.207772,Strike: $457,100,50,1.520461,1.344297,-1.548864,-2.00805,50,100
9,460.499616,Strike: $457,92,58,1.332866,1.044308,-2.132588,-2.552562,42,108


,Underlying,Strike: $458,call_openint 458,call_volume 458,call_bid 458,call_ask 458,put_bid 458,put_ask 458,put_volume 458,put_openint 458
0,452.921909,Strike: $458,108,42,1.979928,1.772897,6.945153,6.805576,58,92
1,453.227405,Strike: $458,96,54,2.21575,2.003712,7.188804,6.596746,46,104
2,456.021882,Strike: $458,109,41,2.356029,1.979743,4.325393,3.969114,59,91
3,456.255263,Strike: $458,107,43,3.374182,3.207266,5.14975,4.909938,57,93
4,457.185308,Strike: $458,111,39,2.80733,2.69563,3.941376,3.347645,61,89
5,458,Strike: $458,124,76,3.441297,3.239906,3.682497,3.133254,0,276
6,458.193019,Strike: $458,108,42,1.677892,1.56307,1.577979,1.111557,58,92
7,458.906949,Strike: $458,117,33,1.69632,1.401533,0.771932,0.259404,67,83
8,460.207772,Strike: $458,107,43,3.148531,2.850962,0.949202,0.609809,57,93
9,460.499616,Strike: $458,128,22,0.174545,-0.001871,-1.934484,-2.624717,78,72


,Underlying,Strike: $459,call_openint 459,call_volume 459,call_bid 459,call_ask 459,put_bid 459,put_ask 459,put_volume 459,put_openint 459
0,452.921909,Strike: $459,155,0,1.703316,1.52676,7.997863,7.523649,105,45
1,453.227405,Strike: $459,155,0,2.083631,1.927858,7.890396,7.49022,105,45
2,456.021882,Strike: $459,138,12,2.524722,2.155591,5.428587,4.91804,88,62
3,456.255263,Strike: $459,152,0,3.472073,3.214455,6.336369,5.772598,102,48
4,457.185308,Strike: $459,175,0,3.20592,3.120942,5.13615,4.76924,125,25
5,458,Strike: $459,136,64,1.964247,1.748055,2.931951,2.605248,0,264
6,458.193019,Strike: $459,194,0,2.021313,1.837648,2.902503,2.497513,144,6
7,458.906949,Strike: $459,135,15,2.508449,2.111121,2.68168,2.323428,85,65
8,460.207772,Strike: $459,162,0,2.516677,2.242242,1.423072,0.912363,112,38
9,460.499616,Strike: $459,137,13,2.755915,2.582637,1.234462,0.911221,87,63


,Underlying,Strike: $460,call_openint 460,call_volume 460,call_bid 460,call_ask 460,put_bid 460,put_ask 460,put_volume 460,put_openint 460
0,452.921909,Strike: $460,141,9,2.123278,1.795853,9.185133,8.835757,91,59
1,453.227405,Strike: $460,140,10,2.553835,2.429925,9.581949,9.14508,90,60
2,456.021882,Strike: $460,141,9,1.205761,1.210945,5.325938,4.847512,91,59
3,456.255263,Strike: $460,146,4,0.069231,-0.053821,4.116048,3.351312,96,54
4,457.185308,Strike: $460,171,0,4.034804,3.864671,6.934237,6.541069,121,29
5,458,Strike: $460,140,60,1.347762,1.266735,3.611299,3.232007,0,260
6,458.193019,Strike: $460,172,0,2.07778,2.000799,4.074109,3.639039,122,28
7,458.906949,Strike: $460,147,3,1.886621,1.676943,3.105292,2.66053,97,53
8,460.207772,Strike: $460,156,0,2.436842,2.27949,2.196199,2.019016,106,44
9,460.499616,Strike: $460,142,8,2.42807,2.456238,2.185351,1.520523,92,58


,Underlying,Strike: $461,call_openint 461,call_volume 461,call_bid 461,call_ask 461,put_bid 461,put_ask 461,put_volume 461,put_openint 461
0,452.921909,Strike: $461,85,65,1.886315,1.802095,10.190407,9.888612,35,115
1,453.227405,Strike: $461,71,79,0.699664,0.530288,8.60251,8.135364,21,129
2,456.021882,Strike: $461,82,68,2.324676,2.186115,7.452424,6.969055,32,118
3,456.255263,Strike: $461,74,76,2.066527,1.803563,6.663538,6.497411,24,126
4,457.185308,Strike: $461,92,58,0.918091,0.668566,4.87065,4.421495,42,108
5,458,Strike: $461,86,114,1.173311,1.014212,4.312026,3.94359,0,314
6,458.193019,Strike: $461,88,62,3.023165,2.981967,6.245648,5.609742,38,112
7,458.906949,Strike: $461,100,50,2.650269,2.422576,4.699967,4.551083,50,100
8,460.207772,Strike: $461,83,67,2.211248,1.952987,3.065166,2.65166,33,117
9,460.499616,Strike: $461,86,64,1.858479,1.391493,2.29864,1.832229,36,114


# Debug v0.1

* fixed call_bid < call_ask
* fixed put_bid < call_ask
* fixed negative call_bid/ask
* sort the dataframe(s) by descending underlying
* #NOTE means code was changed in that line

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rd

cols = ['Underlying']
strikeRange = [455, 456, 457, 458, 459, 460, 461]

for i in range(len(strikeRange)):
    cols += [f'Strike: ${strikeRange[i]}', f'call_openint {strikeRange[i]}', f'call_volume {strikeRange[i]}', f'call_bid {strikeRange[i]}',f'call_ask {strikeRange[i]}',f'put_bid {strikeRange[i]}',f'put_ask {strikeRange[i]}',f'put_volume {strikeRange[i]}',f'put_openint {strikeRange[i]}']

df = pd.DataFrame(columns=cols)

num_trades = 15
for t in range(num_trades):
    if t > 0:
        df.loc[t,'Underlying'] = df.loc[t-1,'Underlying'] + np.random.normal(0,5)
    else:
        df.loc[t,'Underlying'] = 458
        for strike in range(len(strikeRange)):
            df.loc[t,f'call_openint {strikeRange[strike]}'] = round(np.random.normal(100,30))
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(400 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(200 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            
        
    for strike in range(len(strikeRange)):
        df.loc[t,f'Strike: ${strikeRange[strike]}'] = f'Strike: ${strikeRange[strike]}'

        if t>0:
            df.loc[t,f'call_openint {strikeRange[strike]}'] = max(df.loc[t-1,f'call_openint {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(200 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'call_openint {strikeRange[strike]}'],0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(150 - df.loc[t,f'put_openint {strikeRange[strike]}'],0)
            
            old_cOI = df.loc[t-1,f'call_openint {strikeRange[strike]}']
            old_pOI = df.loc[t-1,f'put_openint {strikeRange[strike]}']
            old_cVol = df.loc[t-1,f'call_volume {strikeRange[strike]}']
            old_pVol = df.loc[t-1,f'put_volume {strikeRange[strike]}']

            cOI = df.loc[t,f'call_openint {strikeRange[strike]}']
            pOI = df.loc[t,f'put_openint {strikeRange[strike]}']
            cVol = df.loc[t,f'call_volume {strikeRange[strike]}']
            pVol = df.loc[t,f'put_volume {strikeRange[strike]}']

            call_ask = abs(np.random.normal(2,1)) #NOTE
            call_bid = abs(call_ask-np.random.normal(0.2,0.1)) #NOTE
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            

            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike] #THIS is largely the issue that causes the PCP derived puts be negative

            put_ask = mean_put+np.random.normal(0.2,0.1) #NOTE
            put_bid = mean_put-np.random.normal(0.2,0.1) #NOTE
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask
        
        else:
            call_ask = abs(np.random.normal(2,1)) #NOTE
            call_bid = abs(call_ask-np.random.normal(0.2,0.1)) #NOTE
            df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
            df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
            
            mean_call = (call_bid+call_ask)/2
            mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike] #THIS is largely the issue that causes the PCP derived puts be negative#

            put_ask = mean_put+np.random.normal(0.2,0.1) #NOTE
            put_bid = mean_put-np.random.normal(0.2,0.1) #NOTE
            df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
            df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask

        
df = df.sort_values(by="Underlying").reset_index(drop=True)

cycle = 9
nCycles = len(strikeRange)
underlyingCol = df['Underlying']

df = df.iloc[:, 1:]

for i in range(0, nCycles):
    start = i*cycle
    end = start+cycle
    dfNew = df.iloc[:, start:end]
    dfNew.insert(loc=0, column = "Underlying", value=underlyingCol)
    display(dfNew)


,Underlying,Strike: $455,call_openint 455,call_volume 455,call_bid 455,call_ask 455,put_bid 455,put_ask 455,put_volume 455,put_openint 455
0,429.11308,Strike: $455,37,113,2.357667,2.712899,28.105254,28.641942,0,163
1,429.730423,Strike: $455,16,134,0.866191,1.012021,25.997884,26.462997,0,184
2,430.013733,Strike: $455,34,116,2.249151,2.56461,27.127106,27.408235,0,166
3,431.719932,Strike: $455,38,112,3.152579,3.162376,26.178578,26.748741,0,162
4,431.887026,Strike: $455,18,132,1.368063,1.431086,24.356611,24.722078,0,182
5,432.58706,Strike: $455,47,103,0.874586,1.0798,23.374292,23.658617,0,153
6,432.918934,Strike: $455,38,112,0.96252,1.168087,23.082656,23.449448,0,162
7,433.659488,Strike: $455,22,128,2.843951,2.966674,23.980612,24.380824,0,178
8,445.766608,Strike: $455,33,117,2.958242,3.061933,11.944652,12.55946,0,167
9,446.27894,Strike: $455,40,110,0.404889,0.482626,9.115537,9.467176,0,160


,Underlying,Strike: $456,call_openint 456,call_volume 456,call_bid 456,call_ask 456,put_bid 456,put_ask 456,put_volume 456,put_openint 456
0,429.11308,Strike: $456,83,67,1.867124,2.038247,28.507058,29.072717,33,117
1,429.730423,Strike: $456,106,44,2.90072,3.187888,29.044203,29.518965,56,94
2,430.013733,Strike: $456,91,59,0.832262,0.964277,26.683693,27.000569,41,109
3,431.719932,Strike: $456,95,55,2.712455,2.824801,26.768822,27.341732,45,105
4,431.887026,Strike: $456,107,43,1.819471,2.098408,25.894955,26.20363,57,93
5,432.58706,Strike: $456,83,67,1.60414,1.729241,24.811038,25.398067,33,117
6,432.918934,Strike: $456,77,73,2.306513,2.464856,25.137765,25.653101,27,123
7,433.659488,Strike: $456,108,42,1.169456,1.530825,23.591257,23.75084,58,92
8,445.766608,Strike: $456,110,40,0.624935,0.844339,10.839421,11.263702,60,90
9,446.27894,Strike: $456,80,70,0.56934,0.781243,10.304817,10.623359,30,120


,Underlying,Strike: $457,call_openint 457,call_volume 457,call_bid 457,call_ask 457,put_bid 457,put_ask 457,put_volume 457,put_openint 457
0,429.11308,Strike: $457,113,37,1.49987,1.676117,29.393473,29.668938,63,87
1,429.730423,Strike: $457,166,0,2.900605,3.104743,30.215924,30.743779,116,34
2,430.013733,Strike: $457,119,31,0.189482,0.29345,27.081155,27.369286,69,81
3,431.719932,Strike: $457,144,6,2.352384,2.606733,27.611971,27.947709,94,56
4,431.887026,Strike: $457,161,0,2.666022,2.793891,27.865618,28.10448,111,39
5,432.58706,Strike: $457,112,38,0.072232,0.196898,24.28071,24.877534,62,88
6,432.918934,Strike: $457,120,30,1.218136,1.388421,25.210772,25.561281,70,80
7,433.659488,Strike: $457,139,11,0.799368,1.156673,24.298903,24.633781,89,61
8,445.766608,Strike: $457,168,0,1.664351,1.897016,12.697209,13.225704,118,32
9,446.27894,Strike: $457,108,42,3.450457,3.630767,14.146151,14.405555,58,92


,Underlying,Strike: $458,call_openint 458,call_volume 458,call_bid 458,call_ask 458,put_bid 458,put_ask 458,put_volume 458,put_openint 458
0,429.11308,Strike: $458,122,28,0.872693,1.076554,29.591112,30.018971,72,78
1,429.730423,Strike: $458,106,44,1.514881,1.712717,29.640579,30.214261,56,94
2,430.013733,Strike: $458,121,29,3.31044,3.591038,31.091142,31.672767,71,79
3,431.719932,Strike: $458,108,42,1.114699,1.238403,27.193761,27.665098,58,92
4,431.887026,Strike: $458,118,32,1.518535,1.518597,27.402587,27.996298,68,82
5,432.58706,Strike: $458,111,39,1.36447,1.569902,26.750313,27.225869,61,89
6,432.918934,Strike: $458,132,18,2.28545,2.439409,27.279262,27.562281,82,68
7,433.659488,Strike: $458,112,38,2.086209,2.245317,26.412194,26.874716,62,88
8,445.766608,Strike: $458,109,41,3.456073,3.680058,15.632481,16.093992,59,91
9,446.27894,Strike: $458,129,21,0.189402,0.322322,11.793374,12.062493,79,71


,Underlying,Strike: $459,call_openint 459,call_volume 459,call_bid 459,call_ask 459,put_bid 459,put_ask 459,put_volume 459,put_openint 459
0,429.11308,Strike: $459,136,14,2.29392,2.42972,32.03285,32.555991,86,64
1,429.730423,Strike: $459,147,3,2.912235,3.214222,32.000692,32.611185,97,53
2,430.013733,Strike: $459,143,7,0.71403,0.986663,29.556109,29.919411,93,57
3,431.719932,Strike: $459,145,5,0.548695,0.683879,27.541283,27.944951,95,55
4,431.887026,Strike: $459,143,7,2.351471,2.487658,29.268522,29.736445,93,57
5,432.58706,Strike: $459,142,8,1.24943,1.685265,27.704202,27.99708,92,58
6,432.918934,Strike: $459,136,14,1.634955,1.880309,27.464734,28.068738,86,64
7,433.659488,Strike: $459,146,4,2.464521,2.790035,27.824817,28.035736,96,54
8,445.766608,Strike: $459,149,1,1.05918,1.351969,14.420059,14.565313,99,51
9,446.27894,Strike: $459,151,0,2.486672,2.744538,15.221685,15.46146,101,49


,Underlying,Strike: $460,call_openint 460,call_volume 460,call_bid 460,call_ask 460,put_bid 460,put_ask 460,put_volume 460,put_openint 460
0,429.11308,Strike: $460,135,15,3.007336,3.143595,33.65544,34.29515,85,65
1,429.730423,Strike: $460,155,0,2.988287,3.200914,33.259888,33.744275,105,45
2,430.013733,Strike: $460,149,1,3.628859,3.72481,33.508474,34.061499,99,51
3,431.719932,Strike: $460,150,0,2.549249,2.767513,30.879203,31.124113,100,50
4,431.887026,Strike: $460,136,14,1.526591,1.799329,29.834811,29.887373,86,64
5,432.58706,Strike: $460,142,8,1.12713,1.458261,28.514784,28.957472,92,58
6,432.918934,Strike: $460,141,9,1.837842,1.933244,28.762566,28.972796,91,59
7,433.659488,Strike: $460,160,0,0.556612,0.801085,26.954184,27.234128,110,40
8,445.766608,Strike: $460,150,0,2.6529,2.766,16.732783,16.98605,100,50
9,446.27894,Strike: $460,137,13,2.80494,2.951444,16.39035,16.901461,87,63


,Underlying,Strike: $461,call_openint 461,call_volume 461,call_bid 461,call_ask 461,put_bid 461,put_ask 461,put_volume 461,put_openint 461
0,429.11308,Strike: $461,52,98,2.19913,2.501673,34.089693,34.355244,2,148
1,429.730423,Strike: $461,52,98,0.323137,0.591649,31.625008,31.798793,2,148
2,430.013733,Strike: $461,61,89,2.775057,2.913123,33.529217,33.962165,11,139
3,431.719932,Strike: $461,56,94,0.638126,0.935116,29.856143,30.380942,6,144
4,431.887026,Strike: $461,59,91,0.202632,0.199006,29.236262,29.576674,9,141
5,432.58706,Strike: $461,55,95,1.087287,1.392937,29.510911,29.939721,5,145
6,432.918934,Strike: $461,58,92,3.096521,3.277052,31.210273,31.41015,8,142
7,433.659488,Strike: $461,54,96,0.042307,0.227709,27.363429,27.47871,4,146
8,445.766608,Strike: $461,47,103,3.069941,3.189884,18.049976,18.679942,0,153
9,446.27894,Strike: $461,69,81,0.12447,0.359633,14.901623,15.290481,19,131


It was easy to fix calls because they are our initial conditions that define the system however it is a bit more challanging to work out for puts because puts prices that need to be calibrated based on PCP. Notably puts are fine most of the time but when there is the seed past some particular threshold they seem to go way negative. The numbers aren't consistent.